In [13]:
import os
import requests
from bs4 import BeautifulSoup
from typing import List
from dotenv import load_dotenv
from openai import OpenAI
import google.generativeai
import anthropic
import gradio as gr # oh yeah!

In [14]:
client = OpenAI(base_url="http://localhost:1234/v1", api_key="lm-studio")

In [15]:
system_message = "You are a helpful assistant"

In [16]:
def stream_llama(prompt):
    messages = [
        {"role": "system", "content": system_message},
        {"role": "user", "content": prompt}
      ]
    stream = client.chat.completions.create(
        model='llama-3.2-3b-instruct',
        messages=messages,
        stream=True
    )
    result = ""
    for chunk in stream:
        result += chunk.choices[0].delta.content or ""
        yield result

In [17]:
def stream_phi(prompt):
    messages = [
        {"role": "system", "content": system_message},
        {"role": "user", "content": prompt}
      ]
    stream = client.chat.completions.create(
        model='phi-3.1-mini-128k-instruct',
        messages=messages,
        stream=True
    )
    result = ""
    for chunk in stream:
        result += chunk.choices[0].delta.content or ""
        yield result

In [18]:
def stream_gemma(prompt):
    messages = [
        {"role": "system", "content": system_message},
        {"role": "user", "content": prompt}
      ]
    stream = client.chat.completions.create(
        model='gemma-2-2b-instruct',
        messages=messages,
        stream=True
    )
    result1 = ""
    for chunk in stream:
        result1 += chunk.choices[0].delta.content or ""
        yield result1



In [19]:
def stream_granite(prompt):
    messages = [
        {"role": "system", "content": system_message},
        {"role": "user", "content": prompt}
      ]
    stream = client.chat.completions.create(
        model='granite-3.0-2b-instruct',
        messages=messages,
        stream=True
    )
    result1 = ""
    for chunk in stream:
        result1 += chunk.choices[0].delta.content or ""
        yield result1

In [20]:
def stream_model(prompt, model):
    # print(f"Prompt: {prompt}, Model: {model}")  # Debugging line

    if model is None:
        yield "Error: Model selection is required."
        return

    result = None
    try:
        if model == "Llama_3b (Meta)":
            result = stream_llama(prompt)
        elif model == "Gemma (Google)":
            result = stream_gemma(prompt)
        elif model == "Phi (Microsoft)":
            result = stream_phi(prompt)
        else:
            raise ValueError(f"Unsupported model: {model}")

        for chunk in result:
            yield chunk
    except Exception as e:
        yield f"Error: {str(e)}"

In [29]:
view = gr.Interface(
    fn=stream_model,
    inputs=[
        gr.Textbox(label="Your message:"),
        gr.Dropdown(["Llama_3b (Meta)", "Gemma (Google)", "Phi (Microsoft)"], label="Select model", value="Llama_3b (Meta)")  # Set a default model
    ],
    outputs=[
        gr.Markdown(label="Response:")
    ],
    allow_flagging="never"
)
view.launch( )


C:\Users\iei1\.conda\envs\llms\Lib\site-packages\gradio\interface.py:393: UserWarning: The `allow_flagging` parameter in `Interface` is deprecated.Use `flagging_mode` instead.
  warnings.warn(


* Running on local URL:  http://127.0.0.1:7865

To create a public link, set `share=True` in `launch()`.


# company borchuer

In [22]:
class Website:
    url: str
    title: str
    body: str
    links: List[str]

    def __init__(self, url):
        self.url = url
        response = requests.get(url)
        self.body = response.content
        soup = BeautifulSoup(self.body, 'html.parser')
        self.title = soup.title.string if soup.title else "No title found"
        if soup.body:
            for irrelevant in soup.body(["script", "style", "img", "input"]):
                irrelevant.decompose()
            self.text = soup.body.get_text(separator="\n", strip=True)
        else:
            self.text = ""
        links = [link.get('href') for link in soup.find_all('a')]
        self.links = [link for link in links if link]

    def get_contents(self):
        return f"Webpage Title:\n{self.title}\nWebpage Contents:\n{self.text}\n\n"

In [23]:
system_prompt = "You are an assistant that analyzes the contents of a company website landing page \
and creates a short brochure about the company for prospective customers, investors and recruits. Respond in markdown."

In [24]:
def stream_brochure(company_name, url, model):
    prompt = f"Please generate a company brochure for {company_name}. Here is their landing page:\n"
    prompt += Website(url).get_contents()
    if model == "Llama_3b (Meta)":
        result = stream_llama(prompt)
    elif model == "Gemma (Google)":
        result = stream_gemma(prompt)
    elif model == "Phi (Microsoft)":
        result = stream_phi(prompt)
    else:
        raise ValueError("Unknown model")
    for chunk in result:
        yield chunk

In [25]:
view = gr.Interface(
    fn=stream_brochure,
    inputs=[
        gr.Textbox(label="Company name:"),
        gr.Textbox(label="Landing page URL:"),
        gr.Dropdown(["Llama_3b (Meta)", "Gemma (Google)", "Phi (Microsoft)"], label="Select model", value="Llama_3b (Meta)")],
    outputs=[gr.Markdown(label="Brochure:")],
    allow_flagging="never"
)
view.launch()

* Running on local URL:  http://127.0.0.1:7863

To create a public link, set `share=True` in `launch()`.


In [26]:
def stream_brochure(company_name, url,Tone, model):
    prompt = f"Please generate a company brochure for {company_name}. Here is their landing page:\n"
    prompt += Website(url).get_contents()
    print(f"Received Tone: {Tone}")
    #print(f"Received Tone: {Language}")
   
    tone_prompts = {
        "Casual": (
            "Hey there! You're an assistant that checks out company website landing pages and "
            "whips up a fun, short brochure for potential customers, investors, and job seekers. "
            "Make sure to respond in markdown!"
        ),
        "Formal": (
            "You are an assistant tasked with analyzing the contents of a company's website landing page and "
            "subsequently creating a concise brochure aimed at prospective customers, investors, and recruits. "
            "Please respond in markdown format."
        ),
        "Engaging": (
            "Imagine being an assistant that expertly analyzes a company’s website landing page to craft "
            "a compelling brochure that captivates potential customers, investors, and recruits. "
            "Your responses should be in markdown to enhance clarity and presentation."
        ),
        "Humorous": (
            "You’re the assistant extraordinaire who dives into company website landing pages and magically "
            "transforms them into a short, catchy brochure for curious customers, investors, and job seekers. "
            "Remember, markdown is your secret weapon!"
        ),
        "Inspirational": (
            "As an insightful assistant, you have the power to analyze a company’s website landing page and create "
            "an inspiring brochure that attracts prospective customers, investors, and recruits. "
            "Let your response shine in markdown!"
        )
    }

    # Check if the provided tone is valid
    if Tone not in tone_prompts:
        yield "Error: Unknown tone selected."
        return

    system_prompt = tone_prompts[Tone] ## + f" Please translated english into {Language}."
    print(system_prompt)
    if model == "Llama_3b (Meta)":
        result = stream_llama(prompt)
    elif model == "Gemma (Google)":
        result = stream_gemma(prompt)
    elif model == "Phi (Microsoft)":
        result = stream_phi(prompt)
    elif model == "Granite (IBM)":
        result = stream_granite(prompt)
    else:
        raise ValueError("Unknown model")
    for chunk in result:
        yield chunk

In [28]:
view = gr.Interface(
    fn=stream_brochure,
    inputs=[
        gr.Textbox(label="Company name:"),
        gr.Textbox(label="Landing page URL:"),
        gr.Dropdown(["Casual","Formal", "Engaging","Humorous","Inspirational"], label ="Select Tone", value ="Casual"),
        #gr.Dropdown(["English", "Chinese"], label="Select Language", value="English"),
        gr.Dropdown(["Llama_3b (Meta)", "Gemma (Google)", "Phi (Microsoft)", "Granite (IBM)"], label="Select model", value="Llama_3b (Meta)")],
    outputs=[gr.Markdown(label="Brochure:")],
    allow_flagging="never"
)
view.launch()

* Running on local URL:  http://127.0.0.1:7864

To create a public link, set `share=True` in `launch()`.


Received Tone: Casual
Hey there! You're an assistant that checks out company website landing pages and whips up a fun, short brochure for potential customers, investors, and job seekers. Make sure to respond in markdown!
